In [13]:
import os
import sys
import urllib.request
import datetime
import time
import json
import pandas as pd
import glob
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter

In [17]:
# Naver API Key 가져오기
with open('naver_api_key.txt', 'r') as file:
    lines = file.readlines()
    client_id = lines[0].strip()
    client_secret = lines[1].strip()

In [ ]:
# 저장한 파일 가져오기
kosdaq_day_price_path = './data/day_price/kosdaq'
kospi_day_price_path = './data/day_price/kospi'

kospi_xlsx_files = glob.glob(os.path.join(kospi_day_price_path, '*.xlsx'))
kospi_file_names = [os.path.basename(file) for file in kospi_xlsx_files]

kosdaq_xlsx_files = glob.glob(os.path.join(kosdaq_day_price_path, '*.xlsx'))
kosdaq_file_names = [os.path.basename(file) for file in kosdaq_xlsx_files]

In [21]:
def get_title_keyword(market_list):
    market_name = str(market_list)[:-10]
    save_path = "data/news_and_keyword/" + market_name + "/"
    for i in range(len(market_list)):
        stock_name = market_list[i].replace(".xlsx","")
        #stock_name = '3S'
        node = 'news'  # 크롤링 할 대상
        src_text = '[특징주] ' + stock_name
        cnt = 0
        title_result_list = []
        date_result_list = []
        keyword_result_list = []
        rate_d1_list = []
        rate_d5_list = []
        rate_d20_list = []

        jsonResponse = get_naver_search(node, src_text, 1, 100)  # [CODE 2]
        total = jsonResponse['total']

        while ((jsonResponse != None) and (jsonResponse['display'] != 0)):
            for post in jsonResponse['items']:
                cnt += 1
                title_result, date_result = get_post_data(post, cnt)  # [CODE 3]
                keywords = get_keyword(title_result,stock_name)
                rate_d1, rate_d5, rate_d20 = get_rate(market_name, stock_name, date_result)

                title_result_list.append(title_result)
                date_result_list.append(date_result)
                keyword_result_list.append(keywords)
                rate_d1_list.append(rate_d1)
                rate_d5_list.append(rate_d5)
                rate_d20_list.append(rate_d20)

            start = jsonResponse['start'] + jsonResponse['display']
            print(jsonResponse['display'])
            jsonResponse = get_naver_search(node, src_text, start, 100)  # [CODE 2]
            
        df_news_keyword = pd.DataFrame(list(zip(date_result_list, title_result_list, keyword_result_list)))

        df_news_keyword.to_excel(save_path + stock_name + ".xlsx", index= False)
        print('{}, {} / {}, {}, 뉴스 : {} 건'.foramt(market_name, i+1, len(market_list), stock_name, total))

[2024-07-22 19:41:57.289227] Url Request Success
100
[2024-07-22 19:41:57.439401] Url Request Success
100
[2024-07-22 19:41:57.538183] Url Request Success
9
[2024-07-22 19:41:57.621472] Url Request Success
전체 검색 : 209 건


In [ ]:
for i in range(len(kospi_file_names)):
    #file_path = './data/day_price_kospi/' + kospi_file_names[i]
    stock_name = kospi_file_names[i].replace(".xlsx","")
    
    node = 'news'  # 크롤링 할 대상
    src_text = '[특징주] ' + stock_name
    cnt = 0
    jsonResult = []

    jsonResponse = get_naver_search(node, src_text, 1, 100)  # [CODE 2]
    total = jsonResponse['total']

    while ((jsonResponse != None) and (jsonResponse['display'] != 0)):
        for post in jsonResponse['items']:
            cnt += 1
            get_post_data(post, jsonResult, cnt)  # [CODE 3]

        start = jsonResponse['start'] + jsonResponse['display']
        print(jsonResponse['display'])
        jsonResponse = get_naver_search(node, src_text, start, 100)  # [CODE 2]
        

    print('전체 검색 : %d 건' % total)

In [18]:
# [CODE 1]
def get_request_url(url):
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)

    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            #print("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

In [19]:
# [CODE 2]
def get_naver_search(node, srcText, start, display):
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % node
    parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(srcText), start, display)

    url = base + node + parameters
    responseDecode = get_request_url(url)  # [CODE 1]

    if (responseDecode == None):
        return None
    else:
        return json.loads(responseDecode)

In [20]:
# [CODE 3]
def get_post_data(stock_name, post):
    title_result = ""
    date_result = ""
    
    title = post['title']

    pDate = datetime.datetime.strptime(post['pubDate'], '%a, %d %b %Y %H:%M:%S +0900')
    pDate = pDate.strftime('%Y-%m-%d %H:%M:%S')

    title = title.repalce("<b>","")
    title = title.replace("</b>", "")

    src_text = "[특징주] " + stock_name
    src_text2 = "[특징주]" + stock_name

    if src_text in title or src_text2 in title:
        title_result = title
        date_result = pDate
        
    return title_result, date_result

In [ ]:
def get_keyword(title, stock_name): # 종목명과 기사제목을 받아 키워드를 추출함
    # 직접 정의한 한국어 불용어 목록
    stop_words = set([
        '이', '그', '저', '것', '수', '등', '들', '및', '의', '를', '에', '과', '와', '한', 
        '하다', '되다', '을', '를', '은', '는', '이', '가', '로', '으로', '에게', '께', '에서',
        '이다', '하다', '것', '있다', '되다', '수', '나', '우리', '너', '있다', '이다', '것', '특징주'
    ])
    
    stop_words.add(stock_name) # 불용어 목록에 종목명 추가(기사에 포함된 종목명이 키워드에 걸리지 않기 위해)

    words = word_tokenize(title)

    filtered_words = [word for word in words if word.isalnum() and word not in stop_words] # 불용어 및 특수문자 제거
    word_freq = Counter(filtered_words) # 단어 빈도 계산
    keywords = [word for word, freq in word_freq.most_common(5)] # 키워드 추출 (빈도순 상위 5개 단어)
    return keywords

In [ ]:
def get_rate(market_name, stock_name, date):
    load_path = "./data/day_price/" + market_name + "/" + stock_name + ".xlsx"
    df_stock_price = pd.read_excel(load_path)
    
    base_index = df_stock_price.index[df_stock_price["Date"].str.contains(str(date)[0:10])]
    base_close = df_stock_price["Close"][base_index]

    d1_close = df_stock_price["Close"][base_index + 1]
    d5_close = df_stock_price["Close"][base_index + 5]
    d20_close = df_stock_price["Close"][base_index + 10]
    
    rate_d1 = (d1_close - base_close)/base_close * 100
    rate_d5 = (d5_close - base_close)/base_close * 100
    rate_d20 = (d20_close - base_close)/base_close * 100

    return rate_d1, rate_d5, rate_d20

In [13]:
node = 'news'  # 크롤링 할 대상
src_text = '[특징주] ' + stock_name
cnt = 0
jsonResult = []

jsonResponse = get_naver_search(node, src_text, 1, 100)  # [CODE 2]
total = jsonResponse['total']

while ((jsonResponse != None) and (jsonResponse['display'] != 0)):
    for post in jsonResponse['items']:
        cnt += 1
        get_post_data(post, jsonResult, cnt)  # [CODE 3]

    start = jsonResponse['start'] + jsonResponse['display']
    print(jsonResponse['display'])
    jsonResponse = get_naver_search(node, src_Text, start, 100)  # [CODE 2]
    

print('전체 검색 : %d 건' % total)


#with open('%s_naver_%s.json' % (srcText, node), 'w', encoding='utf8') as outfile:
#    jsonFile = json.dumps(jsonResult, indent=4, sort_keys=True, ensure_ascii=False)

#    outfile.write(jsonFile)

#print("가져온 데이터 : %d 건" % (cnt))
#print('%s_naver_%s.json SAVED' % (srcText, node))

[2024-07-21 17:59:51.919370] Url Request Success
100
[2024-07-21 17:59:52.074942] Url Request Success
100
[2024-07-21 17:59:52.220063] Url Request Success
45
[2024-07-21 17:59:52.307082] Url Request Success
전체 검색 : 245 건
가져온 데이터 : 245 건
[특징주] AJ네트웍스_naver_news.json SAVED
